In [47]:
import os
import json
import traceback
import pandas as pd
from dotenv import load_dotenv


In [46]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain


In [45]:
load_dotenv()#take environment variables from .env

False

In [44]:
import os


In [43]:
key = os.getenv("OPENAI_API_KEY")


In [48]:
print("Value of MY_VARIABLE:", key)

Value of MY_VARIABLE: None
